In [1]:
# Make sure you use the Colab GPU to run the testing phase

import torch
torch.cuda.is_available()

True

In [2]:
# Show the GPU info

!nvidia-smi

Thu Nov 25 02:32:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    31W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install googledrivedownloader

In [5]:
# Import your package and check the version
import torch
import json
import torchvision
import torch
from utils import*
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np
import glob
from tqdm import tqdm
import os
import time
from google_drive_downloader import GoogleDriveDownloader as gdd
print(torch.__version__, torch.cuda.is_available())

1.10.0+cu111 True


In [6]:
# 1. Download the testing data
gdd.download_file_from_google_drive(file_id='1Fm-avdeNgzhPxhvia0iw9yZzcoOggy7I',
                  dest_path='./test.zip',
                  unzip=True)
# 2. Read the image_name and put them into the list
# You need to modify the path to fit your test_folder
data_listdir = os.listdir("test")

Unzipping...Done.


In [7]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

device = torch.device('cuda') # if torch.cuda.is_available() else 'cpu'

#load pretrained_weight
weight_path = '/content/drive/MyDrive/fasterrcnn_resnet50_fpn1.pth'
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(
        pretrained=False,
        progress=True,
        num_classes=11,
        pretrained_backbone=False)    
model.load_state_dict(torch.load(weight_path))
model.to(device)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu

In [8]:
# Test your inference time
TEST_IMAGE_NUMBER = 100  # This number is fixed.
test_img_list = []

transform = transforms.ToTensor()
# Read image (Be careful with the image order)
data_listdir.sort(key=lambda x: int(x[:-4]))
for img_name in data_listdir[:TEST_IMAGE_NUMBER]:
    img_path = os.path.join('test', img_name)
    img = Image.open(img_path).convert("RGB")
    img = transform(img)
    test_img_list.append(img)


start_time = time.time()
for img in tqdm(test_img_list):
    model.eval()
    with torch.no_grad():
        prediction = model([img.to('cuda')])

end_time = time.time()
print("\nInference time per image: ",(end_time - start_time) / len(test_img_list))

# Remember to screenshot!

  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 100/100 [00:29<00:00,  3.36it/s]


Inference time per image:  0.2977695798873901


In [9]:
transform = transforms.ToTensor()
imgs = []  # Stores image names
img_detections = []  # Stores detections for each image index
test_img_list = []
print("\nPerforming object detection:")
data_listdir.sort(key=lambda x: int(x[:-4]))
for img_name in tqdm(data_listdir):
    img_path = os.path.join('test', img_name)
    img = Image.open(img_path).convert("RGB")
    img = transform(img)
    test_img_list.append(img)
    imgs.append(img_name)

for img in tqdm(test_img_list):
    model.eval()
    with torch.no_grad():
        prediction = model([img.to('cuda')])

    # Save detections
    img_detections.extend(prediction)


print("\nSaving images:")
output_list = []
# Iterate through images and save plot of detections
for img_i, (name, detections) in enumerate(zip(imgs, img_detections)):
    img_id = name[:-4]
    if detections is not None:
        for i in range(len(detections['boxes'])):
            if detections['scores'][i].item() < 0.5:
                continue
            dic = {}
            dic["image_id"] = int(img_id)
            dic["score"] = []
            dic["category_id"] = []
            dic["bbox"] = []
            x1 = detections['boxes'][i][0].item()
            y1 = detections['boxes'][i][1].item()
            x2 = detections['boxes'][i][2].item()
            y2 = detections['boxes'][i][3].item()
            width = x2 - x1
            height = y2 - y1
            box = [x1, y1, width, height]
            dic["score"] = detections['scores'][i].item()
            dic["category_id"] = detections['labels'][i].item() % 10
            dic["bbox"] = box
            output_list.append(dic)

# Write the list to answer.json
json_object = json.dumps(output_list, indent=4)

with open("answer.json", "w") as outfile:
    outfile.write(json_object)



Performing object detection:


100%|██████████| 13068/13068 [1:05:39<00:00,  3.32it/s]



Saving images:
